In [1]:
import datetime as dt
from dateutil import parser
import plotly.graph_objects as go
import pandas as pd
import sys
sys.path.append("../")

In [2]:
from api.oanda_api import OandaApi
from infrastructure.instrument_collection import InstrumentCollection as ic
from db.db import DataBaseMongo

In [13]:
database = DataBaseMongo()
icol = ic()
api = OandaApi()

In [6]:
load_icol = icol.DB_LoadInstruments()

In [11]:
pairs = []
for pair, val in icol.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [12]:
pairs

['EUR_GBP',
 'GBP_NZD',
 'GBP_PLN',
 'GBP_CHF',
 'GBP_CAD',
 'GBP_HKD',
 'GBP_ZAR',
 'GBP_JPY',
 'GBP_AUD',
 'GBP_SGD',
 'GBP_USD']

In [16]:
data = []
for p in pairs:
    print(p)
    d_temp = api.get_candles_df(pair_name=p, granularity='D', count=700)
    d_temp['gain'] = ((d_temp.mid_c - d_temp.mid_o) / d_temp.mid_o) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

EUR_GBP
GBP_NZD
GBP_PLN
GBP_CHF
GBP_CAD
GBP_HKD
GBP_ZAR
GBP_JPY
GBP_AUD
GBP_SGD
GBP_USD


In [18]:
candles_df.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
7684,2024-02-05 22:00:00+00:00,107641,1.25368,1.26034,1.25322,1.25978,1.25301,1.26026,1.25279,1.25968,1.25436,1.26041,1.25337,1.25989,0.486568,GBP_USD
7685,2024-02-06 22:00:00+00:00,103924,1.25980,1.26422,1.25945,1.26284,1.25926,1.26413,1.25873,1.26274,1.26033,1.26431,1.25975,1.26293,0.241308,GBP_USD
7686,2024-02-07 22:00:00+00:00,103776,1.26260,1.26390,1.25718,1.26184,1.26197,1.26382,1.25709,1.26173,1.26322,1.26398,1.25727,1.26196,-0.060193,GBP_USD
7687,2024-02-08 22:00:00+00:00,93910,1.26214,1.26432,1.25990,1.26302,1.26129,1.26424,1.25981,1.26294,1.26298,1.26439,1.25998,1.26311,0.069723,GBP_USD
7688,2024-02-11 22:00:00+00:00,79553,1.26269,1.26548,1.26062,1.26282,1.26209,1.26538,1.26053,1.26271,1.26329,1.26558,1.26070,1.26294,0.010295,GBP_USD
